In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_excel('../data_samples/weatherupdated.xlsx')

In [5]:
df.head()

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,NO
1,Sunny,Hot,High,Strong,NO
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [6]:
df.isna().sum()

Outlook        0
Temperature    0
Humidity       0
Wind           0
PlayTennis     0
dtype: int64

In [7]:
df.groupby('Outlook')['PlayTennis'].value_counts()

Outlook   PlayTennis
Overcast  Yes           4
Rain      Yes           3
          NO            2
Sunny     NO            3
          Yes           2
Name: count, dtype: int64

In [8]:
df.groupby('Temperature')['PlayTennis'].value_counts()

Temperature  PlayTennis
Cool         Yes           3
             NO            1
Hot          NO            2
             Yes           2
Mild         Yes           4
             NO            2
Name: count, dtype: int64

In [9]:
df.groupby('Humidity')['PlayTennis'].value_counts()

Humidity  PlayTennis
High      NO            4
          Yes           3
Normal    Yes           6
          NO            1
Name: count, dtype: int64

In [10]:
df.groupby('Wind')['PlayTennis'].value_counts()

Wind    PlayTennis
Strong  NO            3
        Yes           3
Weak    Yes           6
        NO            2
Name: count, dtype: int64

In [11]:
from scipy.stats import entropy

In [12]:
outlook_entropy = (5/14)*entropy([2/5,3/5],base=2)+(4/14)*entropy([4/4,0/4],base=2)+(5/14)*entropy([3/5,2/5],base=2)
Temperature_entropy = (4/14)*entropy([3/4,1/4],base=2)+(4/14)*entropy([2/4,2/4],base=2)+(6/14)*entropy([4/6,2/6],base=2)
Humidity_entropy = (7/14)*entropy([4/7,3/7],base=2)+(7/14)*entropy([6/7,1/7],base=2)
Wind_entropy = (6/14)*entropy([3/6,3/6],base=2)+(8/14)*entropy([6/8,2/8],base=2)

In [13]:
outlook_entropy,Temperature_entropy,Humidity_entropy,Wind_entropy

(0.693536138896192, 0.9110633930116763, 0.7884504573082894, 0.8921589282623617)

In [14]:
df.columns

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis'], dtype='object')

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
df = df.apply(LabelEncoder().fit_transform)

In [17]:
df

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,2,1,0,1,0
1,2,1,0,0,0
2,0,1,0,1,1
3,1,2,0,1,1
4,1,0,1,1,1
5,1,0,1,0,0
6,0,0,1,0,1
7,2,2,0,1,0
8,2,0,1,1,1
9,1,2,1,1,1


In [18]:
x = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = df['PlayTennis']

In [19]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(max_depth=4,criterion='entropy')
dtree.fit(x,y)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [20]:
dtree.score(x,y)

1.0

In [21]:
dtree.predict([[2,1,0,1	]])

/Users/udaykumar/Documents/workspace/understanding-gen-ai-ml/venv/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])

In [23]:
from sklearn import tree
from sklearn.tree import export_text,export_graphviz

In [24]:
result = export_text(dtree,feature_names=['Outlook', 'Temperature', 'Humidity', 'Wind'])
print(result)

|--- Outlook <= 0.50
|   |--- class: 1
|--- Outlook >  0.50
|   |--- Humidity <= 0.50
|   |   |--- Outlook <= 1.50
|   |   |   |--- Wind <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- Wind >  0.50
|   |   |   |   |--- class: 1
|   |   |--- Outlook >  1.50
|   |   |   |--- class: 0
|   |--- Humidity >  0.50
|   |   |--- Wind <= 0.50
|   |   |   |--- Outlook <= 1.50
|   |   |   |   |--- class: 0
|   |   |   |--- Outlook >  1.50
|   |   |   |   |--- class: 1
|   |   |--- Wind >  0.50
|   |   |   |--- class: 1



In [25]:
!pip install dtreeviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 KB 7.3 MB/s eta 0:00:00
You should consider upgrading via the '/Users/udaykumar/Documents/workspace/understanding-gen-ai-ml/venv/bin/python3 -m pip install --upgrade pip' command.


In [26]:
import dtreeviz

In [27]:
viz = dtreeviz.model(dtree,X_train=x,y_train=y,target_name='PlayTennis',feature_names=['Outlook', 'Temperature', 'Humidity', 'Wind'],class_names=[0,1])

In [30]:
import numpy as np
def gini_index(dis):
  prob = np.array(dis)
  return 1 - np.sum(prob**2)

In [31]:
df.groupby('Outlook')['PlayTennis'].value_counts()

Outlook  PlayTennis
0        1             4
1        1             3
         0             2
2        0             3
         1             2
Name: count, dtype: int64

In [32]:
outlook1 = [(2/5,3/5),(4/4,0/4),(3/5,2/5)]
oulook_gini = sum([(5/14)* gini_index(dist) for dist in outlook1])

In [33]:
oulook_gini

0.34285714285714286

In [34]:
Temperature1 = [(2/4,2/4),(3/4,1/4),(4/6,2/6)]
Temperature_gini = sum([(4/14)* gini_index(dist) for dist in Temperature1])
Temperature_gini

0.376984126984127

In [35]:
Humidity1 = [(4/7,3/7),(6/7,1/7)]
Humidity_gini = sum([(7/14)* gini_index(dist) for dist in Humidity1])
Humidity_gini

0.3673469387755103

In [36]:
Wind1 = [(6/8,2/8),(3/6,3/6)]
Wind_gini = sum([(8/14)* gini_index(dist) for dist in Wind1])
Wind_gini

0.5